Notebook version 1.0, 31 Aug 2021. Written by Joona Andersson / CSC - IT Center for Science Ltd. joona.andersson77@gmail.com

Licensed under the MIT license: http://www.opensource.org/licenses/mit-license.php
***

# Optimized Shor's algorithm for N = 15
Here we present an optimized version of Shor's algorithm for factoring the number 15 with a = 7, and perform noise simulations using the circuit. The circuit uses 7 qubits and only 31 gates.

In [ ]:
from qat.lang.AQASM import *
from qat.lang.AQASM.qftarith import QFT
import qat.lang.AQASM.qftarith as qftarith
import numpy as np
from qat.qpus import LinAlg
from qat.quops import QuantumChannelKraus
from qat.hardware.default import HardwareModel
from qat.qpus import NoisyQProc
from qat.quops import ParametricPureDephasing, ParametricAmplitudeDamping
from qat.core.util import statistics
from qat.hardware import GatesSpecification
from itertools import product
import matplotlib.pyplot as plt

In [ ]:
N = 15
a = 7
n = 4
t = 3 # size of the upper register

In [ ]:
def u_generator(n):
    rout = QRoutine()
    wires = rout.new_wires(n)
    for q in range(n):
        rout.apply(X, wires[q])
    rout.apply(SWAP, wires[1], wires[2])
    rout.apply(SWAP, wires[2], wires[3])
    rout.apply(SWAP, wires[0], wires[3])
    return rout
u_gate = AbstractGate('U', [int], circuit_generator=u_generator)

In [ ]:
short_shor = Program()
reg = short_shor.qalloc(n+t)
cbits = short_shor.calloc(n+t)
for q in range(t):
    short_shor.apply(H, reg[q])
short_shor.apply(X, reg[t])
for idx in range(t-1):
    for _ in range(2**idx):
        short_shor.apply(u_gate(n).ctrl(), reg[idx], reg[3:n+t])
QFT(t).dag()(reg[:t])
#short_shor.measure(reg[:t], cbits)
circuit = short_shor.to_circ()
%qatdisplay circuit

The plot below demonstrates that this circuit works correctly.

In [ ]:
qpu = LinAlg()
result = qpu.submit(circuit.to_job(qubits =[0,1,2], nbshots=0))
states = [str(sample.state) for sample in result]
probabilities = [sample.probability for sample in result]

plt.bar(states, probabilities, align='center', alpha=0.5)
plt.xlabel('State of the first qubit')
plt.title('Measurement outcome probabilities')

plt.show()    

print(statistics(circuit))

# Noisy simulation
Let's now observe what happens when we replace perfect gates with imperfect gates and add environmental noise to the circuit.

In [ ]:
shots = 0 ## a constant for the number of shots
sim_method = 'deterministic'  ## 'deterministic' or 'stochastic'
sim_method = 'stochastic'  ## 'deterministic' or 'stochastic'

## Defining imperfect gates

In [ ]:
## Gate times and fidelities

# X gate
X_fidelity = 0.95
X_time = 30

#CNOT gate
CNOT_fidelity = 0.95
CNOT_time = 90

 #C-PH gate
CPH_fidelity = 0.95
CPH_time = 40

# H gate
H_fidelity = 0.95
H_time = 30


C_SWAP_fidelity = 0.95
C_SWAP_time = 90

# Environmental noise parameters
T1 = 5000 # qubit's energy relaxation time
T2 = 5000 # qubit's dephasing time
T_phi = 1/(1/T2 - 1/(2*T1)) # Pure dephasing

In [ ]:
## Kraus operators for noisy gates

# Noisy RX
px = X_fidelity
noisy_X = QuantumChannelKraus([np.sqrt(px)*np.array([[0,1], 
                                                      [1,0]]),
                                                       np.sqrt(1-px)*np.identity(2)], name='Noisy X')
#Noisy CNOT
pcnot = CNOT_fidelity 
noisy_CNOT = QuantumChannelKraus([np.sqrt(pcnot)*np.array([[1, 0, 0, 0],
                                                           [0, 1, 0, 0],
                                                           [0, 0, 1, -1j],
                                                           [0, 0, -1j, 1]]),     
                                                      np.sqrt(1-pcnot)*np.identity(4)],
                                                       name="Noisy CNOT")
#Noisy Hadamard
ph = H_fidelity
noisy_H = QuantumChannelKraus([np.sqrt(ph)*np.array([[1, 1], 
                                                     [1, -1]])/np.sqrt(2), 
                                                   np.sqrt(1-ph)*np.identity(2)],
                                                    name='Noisy H')

#Noisy C-PH
pcph = CPH_fidelity
noisy_C_PH = lambda theta : QuantumChannelKraus([np.sqrt(pcph)* np.array([[1, 0, 0, 0],
                                                                         [0, 1, 0, 0],
                                                                         [0, 0, 1, 0],
                                                             [0, 0, 0, np.exp(theta*1j)] ], dtype=np.complex_),
                                                                np.sqrt(1-pcph)*np.identity(4)])
#Noisy C-SWAP
pcswap = C_SWAP_fidelity
noisy_C_SWAP = QuantumChannelKraus([np.sqrt(pcswap)*np.array([[1,0,0,0,0,0,0,0],
                                                            [0,1,0,0,0,0,0,0],
                                                            [0,0,1,0,0,0,0,0],
                                                            [0,0,0,1,0,0,0,0],
                                                            [0,0,0,0,1,0,0,0],
                                                            [0,0,0,0,0,0,1,0],
                                                            [0,0,0,0,0,1,0,0],
                                                            [0,0,0,0,0,0,0,1]]),
                                                             np.sqrt(1-pcswap)*np.identity(8)],
                                                            name='Noisy C-SWAP')


In [ ]:
gate_times = { 'X': X_time,  'C-SWAP': C_SWAP_time, 'H': H_time,
              'C-PH': CPH_time, 'C-X': CNOT_time }

quantum_channels = { 'X': noisy_X, 'C-SWAP': noisy_C_SWAP,
                    'H': noisy_H, 'C-PH': noisy_C_PH, 'C-X': noisy_CNOT }

gates_spec = GatesSpecification(gate_times, quantum_channels)

## Noisy simulation without environmental noise

In [ ]:
hw_model = HardwareModel(gates_spec, None, idle_noise=None) ## None for now

noisy_qpu = NoisyQProc(hardware_model= hw_model, sim_method=sim_method)

job = circuit.to_job(qubits=[0,1,2], nbshots=shots)

result = noisy_qpu.submit(job)

prob_sum = sum([sample.probability for sample in result]) ## normalize probabilities

noisy_states = [str(sample.state) for sample in result]
noisy_probabilities = [sample.probability/prob_sum for sample in result]

plt.figure(figsize=(10, 4))
plt.bar(noisy_states, noisy_probabilities)
plt.title('Measurement outcome probabilities')

plt.show()

%qatdisplay circuit --hardware hw_model

## Defining environmental noise

In [ ]:
#Amplitude damping characterized by T1
amp_damp = ParametricAmplitudeDamping(T_1=T1)
#Pure dephasing characterized by T_phi
pure_dephasing = ParametricPureDephasing(T_phi = T_phi)

## Amplitude damping channel for two qubits. We build a new 4x4 quantum channel whose Kraus operators
## are pairwise Kronecker products of the Kraus operators of single qubit amplitude damping channels.
## The gate duration of SWAP/C-PH is given as parameter 'tau' to amp_damp.  
two_qbit_amp_damp_CNOT = QuantumChannelKraus([np.kron(k1, k2) 
                                     for k1, k2 in product(amp_damp(tau=CNOT_time).kraus_operators,
                                                           amp_damp(tau=CNOT_time).kraus_operators)])

## Pure dephasing channel for two qubits. The logic is exactly same as above.
two_qbit_pure_deph_CNOT = QuantumChannelKraus([np.kron(k1, k2)
                                     for k1, k2 in product(pure_dephasing(tau=CNOT_time).kraus_operators,
                                                           pure_dephasing(tau=CNOT_time).kraus_operators)])
#-------------------------------------------------------------------------------------
# Quantum channels for C-PH
two_qbit_amp_damp_CPH = QuantumChannelKraus([np.kron(k1, k2) 
                                     for k1, k2 in product(amp_damp(tau=CPH_time).kraus_operators,
                                                           amp_damp(tau=CPH_time).kraus_operators)])


two_qbit_pure_deph_CPH = QuantumChannelKraus([np.kron(k1, k2)
                                     for k1, k2 in product(pure_dephasing(tau=CPH_time).kraus_operators,
                                                           pure_dephasing(tau=CPH_time).kraus_operators)])
#--------------------------------------------------------------------------------------
# C-SWAP is a three-qubit gate and requires an 8x8 quantum channel which we can define similarluy to 
# the 4x4 channels.
three_qbit_amp_damp = QuantumChannelKraus([np.kron(np.kron(k1, k2), k3)
                                     for k1, k2, k3 in product(amp_damp(tau=C_SWAP_time).kraus_operators,
                                                           amp_damp(tau=C_SWAP_time).kraus_operators,
                                                           amp_damp(tau=C_SWAP_time).kraus_operators)])


three_qbit_pure_deph = QuantumChannelKraus([np.kron(np.kron(k1, k2), k3)
                                     for k1, k2, k3 in product(pure_dephasing(tau=C_SWAP_time).kraus_operators,
                                                           pure_dephasing(tau=C_SWAP_time).kraus_operators,
                                                           pure_dephasing(tau=C_SWAP_time).kraus_operators)])

gates_noise = { 'X': lambda : amp_damp(tau = X_time)*pure_dephasing(tau=X_time),
                'C-X': lambda : two_qbit_amp_damp_CNOT*two_qbit_pure_deph_CNOT,
                'C-SWAP': lambda : three_qbit_amp_damp*three_qbit_pure_deph,
                'H': lambda : amp_damp(tau=H_time)*pure_dephasing(tau=H_time),
                'C_PH': lambda : two_qbit_amp_damp_CPH*two_qbit_pure_deph_CPH }

idle_noise = [amp_damp, pure_dephasing]

#Create hardware model with imperfect gates and environmental noise
env_hardware_model = HardwareModel(gates_spec, gate_noise = gates_noise, idle_noise=idle_noise)


In [ ]:
env_qpu = NoisyQProc(hardware_model=env_hardware_model, sim_method=sim_method)

env_result = env_qpu.submit(circuit.to_job(qubits=[0,1,2], nbshots=shots))

prob_sum = sum([sample.probability for sample in env_result]) ## normalize probabilities

env_states = [str(sample.state) for sample in env_result]
env_probabilities = [sample.probability/prob_sum for sample in env_result]


plt.figure(figsize=(10, 4))
plt.bar(env_states, env_probabilities)
plt.title('Measurement outcome probabilities')

plt.show()